In [1]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import KNNImputer

s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

file_key_1 = 'Tabular-Playground-Series/Tabular-Playground-Sep-2022/train.csv'
file_key_2 = 'Tabular-Playground-Series/Tabular-Playground-Sep-2022/test.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

## Reading data-files
train = pd.read_csv(file_content_stream_1)
train['date'] = pd.to_datetime(train['date'], format = '%Y-%m-%d')

test = pd.read_csv(file_content_stream_2)
test['date'] = pd.to_datetime(test['date'], format = '%Y-%m-%d')



Matplotlib is building the font cache; this may take a moment.


# Basic Feature Engineering 

In [2]:
train['weekday'] = train['date'].dt.dayofweek
test['weekday'] = test['date'].dt.dayofweek

train['month'] = train['date'].dt.month_name()
test['month'] = test['date'].dt.month_name()

In [3]:
train.head()

,row_id,date,country,store,product,num_sold,weekday,month
0,0,2017-01-01,Belgium,KaggleMart,Kaggle Advanced Techniques,663,6,January
1,1,2017-01-01,Belgium,KaggleMart,Kaggle Getting Started,615,6,January
2,2,2017-01-01,Belgium,KaggleMart,Kaggle Recipe Book,480,6,January
3,3,2017-01-01,Belgium,KaggleMart,Kaggle for Kids: One Smart Goose,710,6,January
4,4,2017-01-01,Belgium,KaggleRama,Kaggle Advanced Techniques,240,6,January


In [6]:
train_means = train.groupby(['country', 'store', 'product', 'month', 'weekday'])['num_sold'].median().to_dict()
test['num_sold'] = test.set_index(['country', 'store', 'product', 'month', 'weekday']).index.map(train_means.get)
test

,row_id,date,country,store,product,weekday,month,prediction,num_sold
0,70128,2021-01-01,Belgium,KaggleMart,Kaggle Advanced Techniques,4,January,437.0,437.0
1,70129,2021-01-01,Belgium,KaggleMart,Kaggle Getting Started,4,January,311.0,311.0
2,70130,2021-01-01,Belgium,KaggleMart,Kaggle Recipe Book,4,January,279.0,279.0
3,70131,2021-01-01,Belgium,KaggleMart,Kaggle for Kids: One Smart Goose,4,January,444.0,444.0
4,70132,2021-01-01,Belgium,KaggleRama,Kaggle Advanced Techniques,4,January,154.0,154.0
...,...,...,...,...,...,...,...,...,...
17515,87643,2021-12-31,Spain,KaggleMart,Kaggle for Kids: One Smart Goose,4,December,299.0,299.0
17516,87644,2021-12-31,Spain,KaggleRama,Kaggle Advanced Techniques,4,December,93.0,93.0
17517,87645,2021-12-31,Spain,KaggleRama,Kaggle Getting Started,4,December,88.0,88.0
17518,87646,2021-12-31,Spain,KaggleRama,Kaggle Recipe Book,4,December,66.0,66.0


In [7]:
test[['row_id', 'num_sold']].to_csv('submission_median.csv',index = False)